# 課題3：タイタニック号乗客の生存状況の分類モデル作成

本課題では、`titanic` というデータセットを使います。これは、1912年に発生したタイタニック号の沈没事故における乗客の生存状況に関するデータセットです。元々は、[Encyclopedia Titanica](https://www.encyclopedia-titanica.org/)で掲載されたデータと言われており、このデータセットを組み込んだPythonのライブラリも複数あります。

今回は、`seaborn` のライブラリに組み込まれた `titanic` のデータセットを使います。各セルに入っているコメントの下に、実行するコードを記入してください。わからない場合は、ここまでのレッスン内容や各種ライブラリの公式ドキュメントを参照しましょう。

## 1. 必要なライブラリのimport

In [1]:
# 必要なライブラリのimport（変更しないでください）
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.model_selection import train_test_split

# DataFrameですべての列を表示する設定（変更しないでください）
pd.options.display.max_columns = None

## 2. データの読み込み

seabornに添付のデータセットから「titanic」を読み込み、内容を確認します。

In [2]:
# seabornからtitanicのデータセットを読み込む（変更しないでください）
dataset = sns.load_dataset("titanic")

`sns.load_dataset()` で読み込んだデータは、pandasのDataFrameになっています。

In [ ]:
# datasetの先頭5件を確認


### 使用する列の指定

今回は `survived, pclass, sex, age, sibsp, parch, fare, embarked` の列を使用します。

#### 参考:各列の説明

- `survived`: 生存区分（0:死亡, 1:生存）
- `pclass`: チケットクラス
- `sex`: 性別（male:男性, female:女性）
- `age`: 年齢
- `sibsp`: 同乗している兄弟や配偶者の数
- `parch`: 同乗している親や子供の数
- `fare`: 料金
- `embarked`: 乗船した港（頭文字）
- `class`: 客室クラス
- `who`: 性別（man:男性, woman:女性）
- `adult_male`: 成人男性ならTrue
- `deck`: 事故の際にどのデッキにいたか
- `embark_town`: 乗船した港名
- `alive`: 生存区分（no:死亡, yes:生存）
- `alone`: 1人で乗船したか

In [ ]:
# datasetから「survived, pclass, sex, age, sibsp, parch, fare, embarked」の列を取得して
# datasetに代入（上書き）する


In [ ]:
# 改めてdatasetの先頭5件を表示


## 3. データの前処理

### 要約統計量の表示

In [ ]:
# 要約統計量を表示


### 欠損値の確認と補完

In [ ]:
# 各列の欠損値の数を確認


ageの欠損値は平均値で補完します。

In [ ]:
# ageの欠損値を、ageの平均値で補完する


embarkedの欠損値は、もっとも乗船者数の多い港で補完します。

その方法はいくつかありますが、ここではその1つとして、DataFrameの特定の1列（Series）が持つ `value_counts()` メソッドを紹介します。このメソッドを実行すると、その列が持つ値ごとのデータ数がわかります。

参考：[pandas.Series.value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html)

In [ ]:
# 乗船者数の多い港を value_counts メソッドで確認


`values_count()` の結果を見て、もっとも乗船者数の多い港の文字で欠損値を埋めるようにします。

In [ ]:
# embarkedの欠損値をもっとも乗船者数の多い港にて補完


上記の処理により、欠損値がなくなったかを確認しましょう。

In [ ]:
# 欠損値の数を確認し、補完後の欠損値が0であることを確認


### ダミー変数への変換

sexとembarkedをダミー変数に変換します。

In [ ]:
# datasetのsexとembarkedをダミー変数に変換してdataset2に代入する


In [ ]:
# dataset2のデータの最初の5行を表示


## 4. 目的変数と説明変数の選択

ここでは、以下の列を使用します。

- 目的変数: `survived`
- 説明変数: それ以外

dataset2より目的変数と説明変数に該当する列を取得してnumpy配列に変換し、変数YとXに格納します。列の除外には、DataFrameの `drop` を使います。`データフレーム.drop(columns=除外したい列名)` です。

In [ ]:
# Y:目的変数に該当する列


# X:説明変数に該当する列。dataset2からsurvivedを除外


In [ ]:
# YとXの形状を確認


## 5. データの分割

この課題ではホールドアウト法でデータを分割します。

In [ ]:
# X と Y を 機械学習用データとテストデータに7:3に分ける(X_train, X_test, Y_train, Y_test)


In [ ]:
# 機械学習用データを、学習データと検証データに7:3に分ける(X_train, X_valid, Y_train, Y_valid)


In [ ]:
# 形状を確認:X_train, X_valid, X_test, Y_train, Y_valid, Y_test


## 6. モデルの選択

ロジスティック回帰と決定木、ランダムフォレスト、SVMの4つのモデルを作成して比較します。

In [ ]:
# 必要なライブラリの追加import（変更しないでください）
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score

モデルの評価（性能の比較）には、F1値を使ってください。以下には1つだけセルを用意していますが、モデルを4つ作って比較する処理のためにセルを増やしてもかまいません。

In [ ]:
# 4つのモデルを作成し、それぞれのF1値を出力する


## 7. パラメータのチューニング

GridSearchCVを使い、性能の良かったランダムフォレストのパラメータのチューニングを行ないます。パラメータの候補については、レッスン本編を参考にしてください。

In [ ]:
# 必要なライブラリの追加import（変更しないでください）
from sklearn.model_selection import GridSearchCV

In [ ]:
# 性能の良かったモデルを作成


In [ ]:
# パラメータの指定


In [ ]:
# グリッドサーチのオブジェクトを作成


In [ ]:
# データの分割:機械学習用データを学習と検証に分けるのはクロスバリデーションで行ってくれる
# （Xg_train, Xg_test, Yg_train, Yg_test）


In [ ]:
# グリッドサーチを実行する


In [ ]:
# 最適なパラメータを表示


ここで得たパラメータをもとに、モデルを再度作成します。

In [ ]:
# 最適なパラメータによるモデルの作成


# モデルの学習


# モデルの予測


In [ ]:
# F1値の出力


## 8. テストデータによる汎化性能の確認

最後にテストデータでモデルの汎化性能を確認しましょう。

In [ ]:
# テストデータを使って予測を行いF1値を算出
